In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from collections import defaultdict

class CactusStandardizer:
    def __init__(self):
        self.G = None
        self.phi_inv = {}
        self.min_cut_value = None
        self.steps = []  # 存储每一步的图形状态
        self.step_descriptions = []  # 存储每一步的描述
        self.node_colors = {}
        self.edge_colors = {}
        self.node_labels = {}
        self.edge_labels = {}
        self.next_id = 0
    
    def _get_next_id(self):
        """生成唯一节点ID"""
        self.next_id += 1
        return f"n{self.next_id}"
    
    def _add_step(self, description):
        """保存当前步骤的状态和描述"""
        self.steps.append(self.G.copy())
        self.step_descriptions.append(description)
    
    def _visualize(self, title, highlight_nodes=None, highlight_edges=None):
        """可视化当前图状态"""
        plt.figure(figsize=(10, 8))
        
        # 设置节点颜色
        if highlight_nodes:
            node_colors = ['#ff9999' if node in highlight_nodes else '#99ccff' for node in self.G.nodes]
        else:
            node_colors = ['#99ccff' for _ in self.G.nodes]
        
        # 设置边颜色
        if highlight_edges:
            edge_colors = ['red' if edge in highlight_edges else '#cccccc' for edge in self.G.edges]
        else:
            edge_colors = ['#cccccc' for _ in self.G.edges]
        
        # 设置节点标签
        node_labels = {}
        for node in self.G.nodes:
            if node in self.phi_inv and self.phi_inv[node]:
                node_labels[node] = f"{node}\n{self.phi_inv[node]}"
            else:
                node_labels[node] = node
        
        # 设置边标签
        edge_labels = {(u, v): f"{d['weight']}" for u, v, d in self.G.edges(data=True)}
        
        pos = nx.spring_layout(self.G, seed=42)
        nx.draw_networkx_nodes(self.G, pos, node_color=node_colors, node_size=800)
        nx.draw_networkx_edges(self.G, pos, edge_color=edge_colors, width=2)
        nx.draw_networkx_labels(self.G, pos, labels=node_labels, font_size=10)
        nx.draw_networkx_edge_labels(self.G, pos, edge_labels=edge_labels, font_size=9)
        
        plt.title(title, fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def _compute_min_cut(self):
        """计算图的最小割值（简化实现）"""
        # 在实际应用中，这里应使用Stoer-Wagner算法
        # 这里简化处理：取所有边权重的平均值作为最小割值
        weights = [d['weight'] for _, _, d in self.G.edges(data=True)]
        return np.mean(weights) if weights else 1.0
    
    def _merge_multiple_edges(self):
        """合并多重边（二元环）"""
        merged_edges = {}
        edges_to_remove = []
        
        # 识别多重边
        for u, v, data in self.G.edges(data=True):
            if u > v:  # 确保只处理一次
                u, v = v, u
            key = (u, v)
            if key in merged_edges:
                merged_edges[key].append(data['weight'])
            else:
                merged_edges[key] = [data['weight']]
        
        # 合并多重边
        for (u, v), weights in merged_edges.items():
            if len(weights) > 1:
                edges_to_remove.append((u, v))
                new_weight = sum(weights)
                self.G.add_edge(u, v, weight=new_weight)
        
        # 移除旧边（NetworkX会自动处理）
        self._add_step("合并二元环（多重边）")
        self._visualize("步骤: 合并二元环（多重边）")
    
    def _find_cycles(self):
        """查找图中的所有简单环（简化实现）"""
        cycles = []
        visited_edges = set()
        
        # 在实际应用中，这里应使用更健壮的环检测算法
        # 这里简化处理：检测所有三角形和四边形环
        for node in self.G.nodes():
            neighbors = list(self.G.neighbors(node))
            for i in range(len(neighbors)):
                for j in range(i+1, len(neighbors)):
                    n1, n2 = neighbors[i], neighbors[j]
                    if self.G.has_edge(n1, n2):
                        # 找到三角形
                        cycle_edges = {(node, n1), (node, n2), (n1, n2)}
                        if not any(edge in visited_edges for edge in cycle_edges):
                            cycles.append(cycle_edges)
                            visited_edges.update(cycle_edges)
                    else:
                        # 检查四边形
                        for k in range(j+1, len(neighbors)):
                            n3 = neighbors[k]
                            if (self.G.has_edge(n1, n3) and self.G.has_edge(n2, n3)):
                                cycle_edges = {(node, n1), (node, n2), (node, n3), (n1, n3), (n2, n3)}
                                if not any(edge in visited_edges for edge in cycle_edges):
                                    cycles.append(cycle_edges)
                                    visited_edges.update(cycle_edges)
        return cycles
    
    def _separate_cycle_nodes(self, cycles):
        """分离环中的节点"""
        # 保存原始节点到新节点的映射
        node_mapping = {}
        
        for cycle_edges in cycles:
            # 从环边中提取所有节点
            cycle_nodes = set()
            for u, v in cycle_edges:
                cycle_nodes.add(u)
                cycle_nodes.add(v)
            
            highlight_edges = list(cycle_edges)
            self._add_step(f"处理环: {cycle_nodes}")
            self._visualize(f"步骤: 处理环 {cycle_nodes}", highlight_edges=highlight_edges)
            
            for node in cycle_nodes:
                # 创建新节点
                node_prime = self._get_next_id()
                node_double_prime = self._get_next_id()
                
                # 保存映射
                node_mapping[node] = (node_prime, node_double_prime)
                
                # 添加新节点
                self.G.add_node(node_prime)
                self.G.add_node(node_double_prime)
                
                # 添加连接新节点的边
                self.G.add_edge(node_prime, node_double_prime, weight=self.min_cut_value)
                
                # 更新映射函数
                self.phi_inv[node_prime] = set()
                self.phi_inv[node_double_prime] = self.phi_inv.get(node, set())
                
                # 处理原节点的边
                for neighbor in list(self.G.neighbors(node)):
                    edge_data = self.G.get_edge_data(node, neighbor)
                    weight = edge_data['weight'] if edge_data else 1.0
                    
                    # 移除原边
                    self.G.remove_edge(node, neighbor)
                    
                    # 重新连接边
                    if (node, neighbor) in cycle_edges or (neighbor, node) in cycle_edges:
                        # 环边连接到node_prime
                        self.G.add_edge(node_prime, neighbor, weight=weight)
                    else:
                        # 非环边连接到node_double_prime
                        self.G.add_edge(node_double_prime, neighbor, weight=weight)
                
                # 移除原节点
                self.G.remove_node(node)
                if node in self.phi_inv:
                    del self.phi_inv[node]
            
            self._add_step(f"分离环节点完成: {cycle_nodes}")
            self._visualize(f"步骤: 分离环节点 {cycle_nodes}")
    
    def _contract_triangles(self):
        """收缩三元环"""
        triangles_found = True
        
        while triangles_found:
            triangles_found = False
            triangles = []
            
            # 查找所有三角形
            for node in list(self.G.nodes()):
                neighbors = list(self.G.neighbors(node))
                for i in range(len(neighbors)):
                    for j in range(i+1, len(neighbors)):
                        n1, n2 = neighbors[i], neighbors[j]
                        if self.G.has_edge(n1, n2):
                            triangles.append((node, n1, n2))
                            triangles_found = True
            
            # 收缩找到的三角形
            for triangle in triangles:
                u, v, w = triangle
                if not (self.G.has_node(u) and self.G.has_node(v) and self.G.has_node(w)):
                    continue
                
                self._add_step(f"收缩三元环: {triangle}")
                self._visualize(f"步骤: 收缩三元环 {triangle}", highlight_nodes=triangle)
                
                # 创建新节点
                new_node = self._get_next_id()
                self.G.add_node(new_node)
                self.phi_inv[new_node] = set()
                
                # 收集外部边
                external_edges = {}
                for node in triangle:
                    for neighbor in list(self.G.neighbors(node)):
                        if neighbor not in triangle:  # 外部节点
                            edge_data = self.G.get_edge_data(node, neighbor)
                            if edge_data:
                                weight = edge_data['weight']
                                external_edges[neighbor] = weight
                
                # 移除三角形节点
                self.G.remove_nodes_from(triangle)
                for node in triangle:
                    if node in self.phi_inv:
                        del self.phi_inv[node]
                
                # 添加新边
                for neighbor, weight in external_edges.items():
                    self.G.add_edge(new_node, neighbor, weight=weight)
                
                self._add_step(f"三元环收缩完成: {triangle} -> {new_node}")
                self._visualize(f"步骤: 三元环收缩 {triangle} -> {new_node}", 
                              highlight_nodes=[new_node])
        
        if not triangles:
            self._add_step("没有找到三元环")
    
    def _remove_redundant_nodes(self):
        """删除冗余节点（度数为2且不在环上）"""
        removed = True
        
        while removed:
            removed = False
            for node in list(self.G.nodes()):
                if self.G.degree(node) == 2 and (not self.phi_inv.get(node) or len(self.phi_inv[node]) == 0):
                    neighbors = list(self.G.neighbors(node))
                    if len(neighbors) != 2:
                        continue
                    
                    u, v = neighbors
                    # 检查是否在环上（简化处理）
                    if self.G.has_edge(u, v):
                        continue  # 在环上，不删除
                    
                    self._add_step(f"删除冗余节点: {node}")
                    self._visualize(f"步骤: 删除冗余节点 {node}", highlight_nodes=[node])
                    
                    # 获取边权
                    weight1 = self.G.get_edge_data(node, u)['weight']
                    weight2 = self.G.get_edge_data(node, v)['weight']
                    
                    # 删除节点和边
                    self.G.remove_node(node)
                    if node in self.phi_inv:
                        del self.phi_inv[node]
                    
                    # 添加新边
                    self.G.add_edge(u, v, weight=weight1 + weight2)
                    
                    self._add_step(f"冗余节点删除完成: {node}")
                    self._visualize(f"步骤: 删除冗余节点 {node} 完成")
                    removed = True
                    break  # 重新开始循环，因为图结构已改变
    
    def standardize(self, graph, phi_inv):
        """执行标准化算法"""
        # 初始化
        self.G = graph.copy()
        self.phi_inv = phi_inv.copy()
        self.next_id = max(int(n[1:]) for n in self.G.nodes() if n.startswith('n')) + 1 if any(n.startswith('n') for n in self.G.nodes()) else 1
        self.steps = []
        self.step_descriptions = []
        
        # 步骤0: 初始状态
        self._add_step("初始状态")
        self._visualize("初始仙人掌图")
        
        # 步骤1: 计算最小割值
        self.min_cut_value = self._compute_min_cut()
        self._add_step(f"计算最小割值: {self.min_cut_value}")
        self._visualize(f"最小割值: {self.min_cut_value}")
        
        # 步骤2: 合并二元环（多重边）
        self._merge_multiple_edges()
        
        # 步骤3: 查找并处理所有环
        cycles = self._find_cycles()
        if cycles:
            self._separate_cycle_nodes(cycles)
        else:
            self._add_step("没有找到环")
            self._visualize("没有找到环")
        
        # 步骤4: 收缩三元环
        self._contract_triangles()
        
        # 步骤5: 删除冗余节点
        self._remove_redundant_nodes()
        
        # 步骤6: 返回标准化结果
        self._add_step("标准化完成")
        self._visualize("标准化仙人掌图")
        
        return self.G, self.phi_inv
    
    def visualize_steps(self):
        """可视化所有步骤"""
        for i, (graph, desc) in enumerate(zip(self.steps, self.step_descriptions)):
            plt.figure(figsize=(10, 8))
            pos = nx.spring_layout(graph, seed=42)
            
            # 设置节点颜色和标签
            node_colors = []
            node_labels = {}
            for node in graph.nodes():
                if node in self.phi_inv and self.phi_inv.get(node):
                    node_labels[node] = f"{node}\n{self.phi_inv[node]}"
                else:
                    node_labels[node] = node
                node_colors.append('#ff9999' if 'n' in node else '#99ccff')
            
            # 绘制图
            nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=800)
            nx.draw_networkx_edges(graph, pos, edge_color='#cccccc', width=2)
            nx.draw_networkx_labels(graph, pos, labels=node_labels, font_size=10)
            
            # 添加边标签
            edge_labels = {(u, v): f"{d['weight']}" for u, v, d in graph.edges(data=True)}
            nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=9)
            
            plt.title(f"步骤 {i+1}: {desc}", fontsize=14)
            plt.axis('off')
            plt.tight_layout()
            plt.show()

# 示例使用
if __name__ == "__main__":
    # 创建一个简单的仙人掌图
    G = nx.Graph()
    
    # 添加节点和边
    nodes = ['A', 'B', 'C', 'D', 'E', 'F']
    edges = [
        ('B', 'F', 2), ('A', 'C', 3),
        ('B', 'C', 4), ('C', 'D', 1),
        ('D', 'E', 2), ('D', 'F', 3),
        ('E', 'F', 4), ('B', 'E', 2)
    ]
    
    for node in nodes:
        G.add_node(node)
    
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)
    
    # 初始化映射函数（简化）
    phi_inv = {
        'A': {'a'}, 'B': {'b'}, 'C': {'c'},
        'D': {'d'}, 'E': {'e'}, 'F': {'f'}
    }
    
    # 创建标准化器
    standardizer = CactusStandardizer()
    
    # 执行标准化
    standardized_G, standardized_phi = standardizer.standardize(G, phi_inv)
    
    # 可视化所有步骤
    standardizer.visualize_steps()